# 处理几何数据 代码练习
> 本notebook是基于[4.2.7-处理几何数据.md](4.2.7-%B4%A6%C0%ED%BC%B8%BA%CE%CA%FD%BE%DD.md)的代码练习，包含了一个示例：通过excel表格制作分年龄的人口普查要素文件。

In [2]:
import arcpy, os
import numpy as np

我们结合游标和几何令牌探索`Point`要素的坐标：

In [3]:
fc = "resource/data2/中国各城市中心/中国各城市中心.shp"
with arcpy.da.SearchCursor(fc, ["SHAPE@XY", "city"]) as cursor: # 返回几何对象的特定值，此处为xy坐标，元祖格式
    for row in cursor:
        x, y = row[0] # 解开元祖
        city = row[1]
        print(f"{city}中心点坐标: {x}, {y}") # 打印点坐标

全国中心点坐标: 110.71619249900004, 31.09440786400006
北京市中心点坐标: 116.39904202900004, 39.90358526800003
天津市中心点坐标: 117.18386143100008, 39.124546990000056
石家庄市中心点坐标: 114.49643611600004, 38.04481885200005
唐山市中心点坐标: 118.16910027600011, 39.63379526500006
秦皇岛市中心点坐标: 119.5807952890001, 39.94142010500008
邯郸市中心点坐标: 114.48477371100012, 36.61206476000007
邢台市中心点坐标: 114.5029627990001, 37.06773647700004
保定市中心点坐标: 115.47607676500002, 38.866808322000054
张家口市中心点坐标: 114.87797441400005, 40.810582510000074
承德市中心点坐标: 117.93287458400005, 40.97465304200006
沧州市中心点坐标: 116.85170076100007, 38.310055771000066
廊坊市中心点坐标: 116.69847314200001, 39.52279449100007
衡水市中心点坐标: 115.66011277200005, 37.734463041000026
太原市中心点坐标: 112.54295112000011, 37.85648723300005
大同市中心点坐标: 113.28825071400001, 40.08911696500007
阳泉市中心点坐标: 113.57688623900003, 37.86068771600003
长治市中心点坐标: 113.10689011500006, 36.19094445500008
晋城市中心点坐标: 112.845163576, 35.49796199900004
朔州市中心点坐标: 112.42654143700008, 39.33010042900003
晋中市中心点坐标: 112.72998103200007, 37.6958181

我们结合游标和几何令牌探索`Polygon`要素的坐标：

In [4]:
fc = "./resource/第七次人口普查数据/shp/分省份的数据/分年龄、性别的人口_省份等级.shp"

with arcpy.da.SearchCursor(fc, ["省","SHAPE@"]) as cursor: # 使用字符串 "OID@" 引用唯一标识符字段。  
    for row in cursor:
        if row[0] == "北京市":
            # 探索北京市的顶点坐标
            count = 0
            for part in row[1]:
                for pnt in part:
                    count += 1
                    print(f"第{count}个顶点坐标为{pnt.X}, {pnt.Y}")
                print(f"总共有{count}个顶点")

第1个顶点坐标为117.38335014519316, 40.226472314945454
第2个顶点坐标为117.38556534536451, 40.22405173030319
第3个顶点坐标为117.38697686459068, 40.2201720954659
第4个顶点坐标为117.38695820995349, 40.220152554996446
第5个顶点坐标为117.38694822028424, 40.22014282972788
第6个顶点坐标为117.38694021991523, 40.2201355650044
第7个顶点坐标为117.38693159002094, 40.22012824002633
第8个顶点坐标为117.38692309502483, 40.22012152478857
第9个顶点坐标为117.38691046494603, 40.22011232022743
第10个顶点坐标为117.38689809477137, 40.22010405545785
第11个顶点坐标为117.38688549526944, 40.220096320388905
第12个顶点坐标为117.38546616543124, 40.219262854796966
第13个顶点坐标为117.3854561002189, 40.21925705057248
第14个顶点坐标为117.38544618969001, 40.21925150984936
第15个顶点坐标为117.38543633491895, 40.21924620025197
第16个顶点坐标为117.38542637043076, 40.21924112987432
第17个顶点坐标为117.38541515498548, 40.21923574473391
第18个顶点坐标为117.3854048001914, 40.2192310655613
第19个顶点坐标为117.38538968978241, 40.219224684871335
第20个顶点坐标为117.38537423493301, 40.21921869538653
第21个顶点坐标为117.38535852467623, 40.219213025161025
第22个顶点坐标为117.38534255

## 示例2：通过excel表格制作分年龄的人口普查要素文件

### 方法一 通过geopandas 安装方法：conda install geopandas -c esri

In [5]:
# 准备excel文件
import pandas as pd
df = pd.read_excel("./resource/data2/表2 分年龄、性别的人口(全部数据).xlsx")
df

,地区,0岁,Unnamed: 2,1-4岁,Unnamed: 4,5-9岁,Unnamed: 6,10-14岁,Unnamed: 8,15-19岁,...,65-69岁,Unnamed: 30,70-74岁,Unnamed: 32,75-79岁,Unnamed: 34,80-84岁,Unnamed: 36,85岁及以上,Unnamed: 38
0,NaN,男,女,男,女,男,女,男,女,男,...,男,女,男,女,男,女,男,女,男,女
1,合计,6312409,5675648,34656922,31238909,48017458,42226598,45606790,39649204,39053343,...,36337923,37667637,24162733,25427303,14752433,16486416,9157003,11225875,6100269,9317688
2,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
3,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
4,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,铁门关市,271,266,1648,1542,2497,2290,2561,2246,2107,...,1428,1509,993,1570,1099,1517,990,1074,530,595
3361,双河市,191,203,1093,1030,1450,1415,1613,1341,1626,...,726,757,407,769,504,759,560,418,189,200
3362,可克达拉市,199,197,1126,1063,1772,1702,1926,1839,2045,...,1141,1170,753,1275,953,1421,913,760,334,410
3363,昆玉市,187,201,1642,1487,4112,3755,2516,2260,2203,...,534,569,377,339,210,237,138,124,93,87


原始表格，此表格无港澳台人口信息：
![表2 分年龄、性别的人口(全部数据).xlsx](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308122146861.png)

In [6]:
# 初步处理此dataframe
# 删除空值
df = df.dropna(axis=0, how='all') 
df

,地区,0岁,Unnamed: 2,1-4岁,Unnamed: 4,5-9岁,Unnamed: 6,10-14岁,Unnamed: 8,15-19岁,...,65-69岁,Unnamed: 30,70-74岁,Unnamed: 32,75-79岁,Unnamed: 34,80-84岁,Unnamed: 36,85岁及以上,Unnamed: 38
0,NaN,男,女,男,女,男,女,男,女,男,...,男,女,男,女,男,女,男,女,男,女
1,合计,6312409,5675648,34656922,31238909,48017458,42226598,45606790,39649204,39053343,...,36337923,37667637,24162733,25427303,14752433,16486416,9157003,11225875,6100269,9317688
2,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
3,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
4,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,铁门关市,271,266,1648,1542,2497,2290,2561,2246,2107,...,1428,1509,993,1570,1099,1517,990,1074,530,595
3361,双河市,191,203,1093,1030,1450,1415,1613,1341,1626,...,726,757,407,769,504,759,560,418,189,200
3362,可克达拉市,199,197,1126,1063,1772,1702,1926,1839,2045,...,1141,1170,753,1275,953,1421,913,760,334,410
3363,昆玉市,187,201,1642,1487,4112,3755,2516,2260,2203,...,534,569,377,339,210,237,138,124,93,87


In [7]:
# 去除第一行
df = df.drop(1, axis=0).reset_index(drop=True)
df.head()

,地区,0岁,Unnamed: 2,1-4岁,Unnamed: 4,5-9岁,Unnamed: 6,10-14岁,Unnamed: 8,15-19岁,...,65-69岁,Unnamed: 30,70-74岁,Unnamed: 32,75-79岁,Unnamed: 34,80-84岁,Unnamed: 36,85岁及以上,Unnamed: 38
0,NaN,男,女,男,女,男,女,男,女,男,...,男,女,男,女,男,女,男,女,男,女
1,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
2,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
3,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
4,西城区,3635,3339,20958,19709,35042,31641,22773,20815,13800,...,37033,41923,20204,22432,11531,14168,10271,15502,11907,16061


In [8]:
# 重命名列名 此出直接指定列名
fields = [
    "Province",
    "M0",
    "F0",
    "M1_4",
    "F1_4",
    "M5_9",
    "F5_9",
    "M10_14",
    "F10_14",
    "M15_19",
    "F15_19",
    "M20_24",
    "F20_24",
    "M25_29",
    "F25_29",
    "M30_34",
    "F30_34",
    "M35_39",
    "F35_39",
    "M40_44",
    "F40_44",
    "M45_49",
    "F45_49",
    "M50_54",
    "F50_54",
    "M55_59",
    "F55_59",
    "M60_64",
    "F60_64",
    "M65_69",
    "F65_69",
    "M70_74",
    "F70_74",
    "M75_79",
    "F75_79",
    "M80_84",
    "F80_84",
    "M85+",
    "F85+"
]
df.columns = fields

In [9]:
df.head()

,Province,M0,F0,M1_4,F1_4,M5_9,F5_9,M10_14,F10_14,M15_19,...,M65_69,F65_69,M70_74,F70_74,M75_79,F75_79,M80_84,F80_84,M85+,F85+
0,NaN,男,女,男,女,男,女,男,女,男,...,男,女,男,女,男,女,男,女,男,女
1,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
2,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
3,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
4,西城区,3635,3339,20958,19709,35042,31641,22773,20815,13800,...,37033,41923,20204,22432,11531,14168,10271,15502,11907,16061


In [10]:
df = df.drop(0, axis=0).reset_index(drop=True)
df.head()

,Province,M0,F0,M1_4,F1_4,M5_9,F5_9,M10_14,F10_14,M15_19,...,M65_69,F65_69,M70_74,F70_74,M75_79,F75_79,M80_84,F80_84,M85+,F85+
0,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
1,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
2,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
3,西城区,3635,3339,20958,19709,35042,31641,22773,20815,13800,...,37033,41923,20204,22432,11531,14168,10271,15502,11907,16061
4,朝阳区,12329,11304,66048,61070,76137,71025,50283,46996,41172,...,89009,101817,50634,59091,32240,40815,29164,38203,23785,28017


In [11]:
# 要做的就是把人口数据全部通过地区字段和省份地图匹配 
import geopandas as gpd
gdf = gpd.read_file("./resource/data2/中国分省份空地图/中国各省份地图.shp", encoding="utf-8")
gdf.head()

,省,省级码,geometry
0,北京市,110000,"POLYGON ((117.38335 40.22647, 117.38557 40.224..."
1,天津市,120000,"POLYGON ((117.56937 40.19153, 117.56744 40.189..."
2,河北省,130000,"MULTIPOLYGON (((118.26945 38.98097, 118.26871 ..."
3,山西省,140000,"POLYGON ((114.13714 40.73445, 114.13860 40.732..."
4,内蒙古自治区,150000,"POLYGON ((121.49813 53.32607, 121.50116 53.321..."


In [12]:
gdf_new = pd.merge(gdf, df, left_on="省", right_on="Province", how="left").drop("Province", axis=1)
gdf_new.head()

,省,省级码,geometry,M0,F0,M1_4,F1_4,M5_9,F5_9,M10_14,...,M65_69,F65_69,M70_74,F70_74,M75_79,F75_79,M80_84,F80_84,M85+,F85+
0,北京市,110000,"POLYGON ((117.38335 40.22647, 117.38557 40.224...",79388,73342,447165,416355,485161,447792,335714,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
1,天津市,120000,"POLYGON ((117.56937 40.19153, 117.56744 40.189...",43247,39802,272661,253305,350320,314666,316570,...,417479,443253,251367,269201,140779,160876,88866,109030,71210,93631
2,河北省,130000,"MULTIPOLYGON (((118.26945 38.98097, 118.26871 ...",316473,290961,1869601,1715582,2975262,2614026,2826356,...,2118493,2215930,1369966,1495659,721592,825659,428901,527035,265129,419573
3,山西省,140000,"POLYGON ((114.13714 40.73445, 114.13860 40.732...",147446,140301,767699,729614,1009444,939984,1023296,...,914113,909574,581441,598387,339271,366699,215588,253613,136350,189638
4,内蒙古自治区,150000,"POLYGON ((121.49813 53.32607, 121.50116 53.321...",89403,83548,456610,426095,616732,567333,592970,...,645995,678268,376914,409266,220192,261066,154556,184935,94244,113482


In [13]:
gdf_new.tail()

,省,省级码,geometry,M0,F0,M1_4,F1_4,M5_9,F5_9,M10_14,...,M65_69,F65_69,M70_74,F70_74,M75_79,F75_79,M80_84,F80_84,M85+,F85+
29,新疆维吾尔自治区,650000,"POLYGON ((87.79720 49.18060, 87.81916 49.17268...",92597,86127,596218,560675,1306278,1237342,991442,...,352686,366298,242125,282936,173453,211851,123657,123562,62484,66833
30,江苏省,320000,"MULTIPOLYGON (((121.56617 32.22928, 121.56693 ...",293718,269494,1702238,1553177,2518917,2193839,2360131,...,2537937,2571942,1739491,1807185,1102639,1214600,680452,839548,462422,770315
31,台湾省,710000,"MULTIPOLYGON (((123.69793 25.92930, 123.69726 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,香港特别行政区,810000,"MULTIPOLYGON (((114.22665 22.54375, 114.22661 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,澳门特别行政区,820000,"MULTIPOLYGON (((113.55346 22.21547, 113.55374 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
gdf_new.dtypes

省             object
省级码            int64
geometry    geometry
M0            object
F0            object
M1_4          object
F1_4          object
M5_9          object
F5_9          object
M10_14        object
F10_14        object
M15_19        object
F15_19        object
M20_24        object
F20_24        object
M25_29        object
F25_29        object
M30_34        object
F30_34        object
M35_39        object
F35_39        object
M40_44        object
F40_44        object
M45_49        object
F45_49        object
M50_54        object
F50_54        object
M55_59        object
F55_59        object
M60_64        object
F60_64        object
M65_69        object
F65_69        object
M70_74        object
F70_74        object
M75_79        object
F75_79        object
M80_84        object
F80_84        object
M85+          object
F85+          object
dtype: object

In [15]:
# 将数据列转化为整数

In [16]:
cols_to_convert = gdf_new.columns[3:] 

gdf_new[cols_to_convert] = gdf_new[cols_to_convert].apply(lambda x : x.astype(int) if not pd.isnull else x)   

gdf_new.dtypes

省             object
省级码            int64
geometry    geometry
M0            object
F0            object
M1_4          object
F1_4          object
M5_9          object
F5_9          object
M10_14        object
F10_14        object
M15_19        object
F15_19        object
M20_24        object
F20_24        object
M25_29        object
F25_29        object
M30_34        object
F30_34        object
M35_39        object
F35_39        object
M40_44        object
F40_44        object
M45_49        object
F45_49        object
M50_54        object
F50_54        object
M55_59        object
F55_59        object
M60_64        object
F60_64        object
M65_69        object
F65_69        object
M70_74        object
F70_74        object
M75_79        object
F75_79        object
M80_84        object
F80_84        object
M85+          object
F85+          object
dtype: object

In [17]:
# 保存为shp文件

In [18]:
import os
if os.path.exists("./resource/data2/output") == False:
    os.mkdir("./resource/data2/output")
gdf_new.to_file("./resource/data2/output/分年龄、性别的人口_省份等级_方法1.shp", encoding="utf-8", driver="ESRI Shapefile",  engine="pyogrio") # pip安装pyogrio

出现了列名编码问题！！老老实实把字段写成英文

### 使用arcpy 方法二
此方法如果只用arcpy的游标更新数据，会很麻烦，试了一下放弃了，用pandas的merge的方法会快一万倍。


In [1]:
import arcpy
import pandas as pd
import os

In [2]:
arcpy.env.workspace = "./resource/data2"
arcpy.env.overwriteOutput = True

In [20]:
# 继续用这个表格 
df.head()

,Province,M0,F0,M1_4,F1_4,M5_9,F5_9,M10_14,F10_14,M15_19,...,M65_69,F65_69,M70_74,F70_74,M75_79,F75_79,M80_84,F80_84,M85+,F85+
0,北京市,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
1,市辖区,79388,73342,447165,416355,485161,447792,335714,306590,345715,...,568385,626286,314838,353703,184665,230508,152087,196699,124749,160140
2,东城区,2265,2126,13193,12455,21577,19742,13924,13008,8768,...,24441,27308,13244,14362,7332,8981,6550,10128,6931,9972
3,西城区,3635,3339,20958,19709,35042,31641,22773,20815,13800,...,37033,41923,20204,22432,11531,14168,10271,15502,11907,16061
4,朝阳区,12329,11304,66048,61070,76137,71025,50283,46996,41172,...,89009,101817,50634,59091,32240,40815,29164,38203,23785,28017


In [21]:

# 创建一个数据库来操作 避免覆盖
if os.path.exists("./resource/data2/output.gdb") == False:
    arcpy.management.CreateFileGDB("./resource/data2", "output.gdb")
# 1.选择空的分省份地图要素
fc = "./resource/data2/中国分省份空地图/中国各省份地图.shp"
# 复制到数据库
arcpy.conversion.FeatureClassToFeatureClass(fc, "./resource/data2/output.gdb", "分年龄、性别的人口_省份等级_方法2")

<Result './resource/data2/output.gdb\\分年龄、性别的人口_省份等级_方法2'>

空地图：
![fc](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308122308005.png)

In [22]:
# 3.添加人口字段
new_field = df.columns[1:].tolist()
new_field

['M0',
 'F0',
 'M1_4',
 'F1_4',
 'M5_9',
 'F5_9',
 'M10_14',
 'F10_14',
 'M15_19',
 'F15_19',
 'M20_24',
 'F20_24',
 'M25_29',
 'F25_29',
 'M30_34',
 'F30_34',
 'M35_39',
 'F35_39',
 'M40_44',
 'F40_44',
 'M45_49',
 'F45_49',
 'M50_54',
 'F50_54',
 'M55_59',
 'F55_59',
 'M60_64',
 'F60_64',
 'M65_69',
 'F65_69',
 'M70_74',
 'F70_74',
 'M75_79',
 'F75_79',
 'M80_84',
 'F80_84',
 'M85+',
 'F85+']

In [23]:
# 使用插入游标添加字段

In [24]:
for field in new_field:
    arcpy.management.AddField("output.gdb/分年龄、性别的人口_省份等级_方法2", field, "LONG")

In [25]:
# 列出字段 字段名被arcgis规整化了
gis_field = [field.name for field in arcpy.ListFields("output.gdb/分年龄、性别的人口_省份等级_方法2")]
gis_field = gis_field[2:3] + gis_field[6:]
gis_field

['省',
 'M0',
 'F0',
 'M1_4',
 'F1_4',
 'M5_9',
 'F5_9',
 'M10_14',
 'F10_14',
 'M15_19',
 'F15_19',
 'M20_24',
 'F20_24',
 'M25_29',
 'F25_29',
 'M30_34',
 'F30_34',
 'M35_39',
 'F35_39',
 'M40_44',
 'F40_44',
 'M45_49',
 'F45_49',
 'M50_54',
 'F50_54',
 'M55_59',
 'F55_59',
 'M60_64',
 'F60_64',
 'M65_69',
 'F65_69',
 'M70_74',
 'F70_74',
 'M75_79',
 'F75_79',
 'M80_84',
 'F80_84',
 'M85_',
 'F85_']

In [38]:
# 使用游标添加数据 通过省份字段匹配
# search_field = ["省"] + gis_field
with arcpy.da.UpdateCursor("output.gdb/分年龄、性别的人口_省份等级_方法2", gis_field
) as cursor:
    for i, row in enumerate(cursor):
        # 通过游标所在行的"省"字段匹配df中"Province"的字段
        filtered_df = df[df['Province'] == row[0]]
        if filtered_df.empty:
            print("不匹配")
        else:
            print("匹配")
            print(filtered_df)
            # 赋值
            
            for j, field in enumerate(gis_field[1:]):
                # 因为gis把字段规整化了 有两个例外我们替换一下
                if field == "F85_":
                    field = "F85+"
                elif field == "M85_":
                    field = "M85+"
                
                print(field)
                print(filtered_df.loc[:, field].values[0])
                
                row[j+1] = filtered_df.loc[:, field].values[0] # 这里的j+1是因为gis_field中去掉了"省"字段
            cursor.updateRow(row)
            # break # 测试用

匹配
  Province     M0     F0    M1_4    F1_4    M5_9    F5_9  M10_14  F10_14  \
0      北京市  79388  73342  447165  416355  485161  447792  335714  306590   

   M15_19  ...  M65_69  F65_69  M70_74  F70_74  M75_79  F75_79  M80_84  \
0  345715  ...  568385  626286  314838  353703  184665  230508  152087   

   F80_84    M85+    F85+  
0  196699  124749  160140  

[1 rows x 39 columns]
M0
79388
F0
73342
M1_4
447165
F1_4
416355
M5_9
485161
F5_9
447792
M10_14
335714
F10_14
306590
M15_19
345715
F15_19
287842
M20_24
715999
F20_24
634503
M25_29
995896
F25_29
908792
M30_34
1309189
F30_34
1193840
M35_39
1108136
F35_39
1035049
M40_44
838444
F40_44
763689
M45_49
842337
F45_49
776237
M50_54
842484
F50_54
777307
M55_59
828013
F55_59
799526
M60_64
677025
F60_64
709505
M65_69
568385
F65_69
626286
M70_74
314838
F70_74
353703
M75_79
184665
F75_79
230508
M80_84
152087
F80_84
196699
M85+
124749
F85+
160140
匹配
   Province     M0     F0    M1_4    F1_4    M5_9    F5_9  M10_14  F10_14  \
18      天津市  43247  39

运行完整之后和原始df对比一下：
![](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308130941323.png)

In [33]:
df.iloc[0, :]

Province        北京市
M0            79388
F0            73342
M1_4         447165
F1_4         416355
M5_9         485161
F5_9         447792
M10_14       335714
F10_14       306590
M15_19       345715
F15_19       287842
M20_24       715999
F20_24       634503
M25_29       995896
F25_29       908792
M30_34      1309189
F30_34      1193840
M35_39      1108136
F35_39      1035049
M40_44       838444
F40_44       763689
M45_49       842337
F45_49       776237
M50_54       842484
F50_54       777307
M55_59       828013
F55_59       799526
M60_64       677025
F60_64       709505
M65_69       568385
F65_69       626286
M70_74       314838
F70_74       353703
M75_79       184665
F75_79       230508
M80_84       152087
F80_84       196699
M85+         124749
F85+         160140
Name: 0, dtype: object

In [41]:
# 更改别名
alias = [
    "男性0岁人口",
    "女性0岁人口",
    "男性1-4岁人口",
    "女性1-4岁人口",
    "男性5-9岁人口",
    "女性5-9岁人口",
    "男性10-14岁人口",
    "女性10-14岁人口",
    "男性15-19岁人口",
    "女性15-19岁人口",
    "男性20-24岁人口",
    "女性20-24岁人口",
    "男性25-29岁人口",
    "女性25-29岁人口",
    "男性30-34岁人口",
    "女性30-34岁人口",
    "男性35-39岁人口",
    "女性35-39岁人口",
    "男性40-44岁人口",
    "女性40-44岁人口",
    "男性45-49岁人口",
    "女性45-49岁人口",
    "男性50-54岁人口",
    "女性50-54岁人口",
    "男性55-59岁人口",
    "女性55-59岁人口",
    "男性60-64岁人口",
    "女性60-64岁人口",
    "男性65-69岁人口",
    "女性65-69岁人口",
    "男性70-74岁人口",
    "女性70-74岁人口",
    "男性75-79岁人口",
    "女性75-79岁人口",
    "男性80-84岁人口",
    "女性80-84岁人口",
    "男性85岁以上人口",
    "女性85岁以上人口"
]

for i, field in enumerate(gis_field[1:]): 
    print(field)
    print(alias[i])
    arcpy.management.AlterField("output.gdb/分年龄、性别的人口_省份等级_方法2", field, "" , alias[i])

M0
男性0岁人口
F0
女性0岁人口
M1_4
男性1-4岁人口
F1_4
女性1-4岁人口
M5_9
男性5-9岁人口
F5_9
女性5-9岁人口
M10_14
男性10-14岁人口
F10_14
女性10-14岁人口
M15_19
男性15-19岁人口
F15_19
女性15-19岁人口
M20_24
男性20-24岁人口
F20_24
女性20-24岁人口
M25_29
男性25-29岁人口
F25_29
女性25-29岁人口
M30_34
男性30-34岁人口
F30_34
女性30-34岁人口
M35_39
男性35-39岁人口
F35_39
女性35-39岁人口
M40_44
男性40-44岁人口
F40_44
女性40-44岁人口
M45_49
男性45-49岁人口
F45_49
女性45-49岁人口
M50_54
男性50-54岁人口
F50_54
女性50-54岁人口
M55_59
男性55-59岁人口
F55_59
女性55-59岁人口
M60_64
男性60-64岁人口
F60_64
女性60-64岁人口
M65_69
男性65-69岁人口
F65_69
女性65-69岁人口
M70_74
男性70-74岁人口
F70_74
女性70-74岁人口
M75_79
男性75-79岁人口
F75_79
女性75-79岁人口
M80_84
男性80-84岁人口
F80_84
女性80-84岁人口
M85_
男性85岁以上人口
F85_
女性85岁以上人口


更改别名后的属性表：![更改别名后的属性表](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308130951676.png)

In [6]:
##### 5.导出为shp文件
in_features = "output.gdb/分年龄、性别的人口_省份等级_方法2"
out_features = "output/分年龄、性别的人口_省份等级_方法2.shp"

arcpy.conversion.ExportFeatures(in_features, out_features)

<Result './resource/data2\\output\\分年龄、性别的人口_省份等级_方法2_USE_ALIAS.shp'>